# ter
> TER Translation Error Rate automated machine translation evaluation

> Matthew Snover, Bonnie Dorr, Richard Schwartz, Linnea Micciulla, and John Makhoul, "A Study of Translation Edit Rate with Targeted Human Annotation," Proceedings of Association for Machine Translation in the Americas, 2006. 

In [ ]:
#| default_exp ter

In [ ]:
#| hide
import os

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
homedir = ""
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')

In [ ]:
#| hide
if running_in_colab:
    GITHUB_TEST_FOLDER = homedir+"/github/polyglottech/mteval"
    %cd {GITHUB_TEST_FOLDER}
    !pip3 install -e '.[dev]'

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from sacrebleu.metrics import TER
def measure_ter(hypothesis_lines,reference_lines,normalized=False,no_punct=False,asian_support=False,case_sensitive=False):
    """Measuring TER on set of hypotheses and references
    
    Parameters
    ----------
    hypothesis_lines : List 
        Array of hypotheses.
    reference_lines : List
        Array of references.
    normalized : bool, default=False
        If `True`, applies basic tokenization to sentences.
    no_punct : bool, default=False
        If `True`, removes punctuations from sentences.
    asian_support : bool, default=False
        If `True`, adds support for Asian character processing.
    case_sensitive : bool, default=False
        If `True`, does not lowercase sentences.
    
    Returns
    -------
    Tuple
        Array of arrays containing TER score, number of edits and reference length and sacreBLEU TER score in JSON format.
    """
    # Calculate corpus level score
    ter = TER(normalized=normalized,no_punct=no_punct,asian_support=asian_support,case_sensitive=case_sensitive)
    corpus_score = ter.corpus_score(hypothesis_lines,[reference_lines])
    sig = ter.get_signature().format(short=False)
    score_json = corpus_score.format(width=2,score_only=False,signature=sig,is_json=True)
    
    # Calculate segment level scores
    seg_scores = []
    for hypothesis, reference in zip(hypothesis_lines,reference_lines):
        seg_score = ter.sentence_score(hypothesis,[reference])
        seg_scores.append([seg_score.score,seg_score.num_edits,seg_score.ref_length])
    return seg_scores, score_json

In [ ]:
#| export
import csv
import json
import sys
import os
from pathlib import Path
def measure_record_ter(hypothesis_lines,reference_lines,sourcelang,targetlang,test_set_name,test_date,mtengine,score_pathname,score_fname,domain='',normalized=False,no_punct=False,asian_support=False,case_sensitive=False):
    """Score hypothesis with TER score and record it to a specified metrics file
    
    Parameters
    ----------
    hypothesis_lines : List 
        Array of hypotheses.
    reference_lines : List
        Array of references.
    sourcelang : str
        Source language identifier.
    targetlang : str
        Target language identifier.
    test_set_name : str
        Name of test set.
    test_date : str
        Date of test as ISO-8601 formatted string.
    mtengine : str
        Name of the tested MT engine.
    score_pathname : str
        Path for results CSV file.
    score_fname : str
        File name for results CSV file.
    domain : str, default=''
        Domain string if applicable.
    normalized : bool, default=False
        If `True`, applies basic tokenization to sentences.
    no_punct : bool, default=False
        If `True`, removes punctuations from sentences.
    asian_support : bool, default=False
        If `True`, adds support for Asian character processing.
    case_sensitive : bool, default=False
        If `True`, does not lowercase sentences.
    """
    seg_scores, score_json = measure_ter(hypothesis_lines,reference_lines,normalized,no_punct,asian_support,case_sensitive)

    score_dict = json.loads(score_json)
    metric_record = score_dict
    metric_record["date"] = test_date
    metric_record["source_langid"] = sourcelang
    metric_record["target_langid"] = targetlang
    metric_record["test_set"] = test_set_name
    metric_record["engine"] = mtengine
    metric_record["domain"] = domain
    score_path = Path(score_pathname,score_fname)
    with open(score_path,"a",newline="") as score_file:
        field_names = metric_record.keys()
        writer = csv.DictWriter(score_file, fieldnames=field_names)
        if score_file.tell() == 0:
            writer.writeheader()
        writer.writerow(metric_record)
    
    # Write segment level TER scores to plain text file
    output_path = score_pathname+"/"+sourcelang+"_"+targetlang+"/"+test_date+"/"+test_set_name
    os.makedirs(output_path,exist_ok=True)
    seg_score_fname = ""
    if domain:
        seg_score_fname = "ter_"+mtengine+"."+domain+"."+sourcelang+"-"+targetlang
    else:
        seg_score_fname = "ter_"+mtengine+"."+sourcelang+"-"+targetlang
    seg_score_path = Path(output_path,seg_score_fname)
    with open(seg_score_path,"w",newline="") as seg_score_fh:
        writer = csv.writer(seg_score_fh)
        for seg_score in seg_scores:
            writer.writerow(seg_score)
    

In [ ]:
#| hide
from mteval.dataset import *
import json
import os
from datetime import date

In [ ]:
#| hide
mteval_test_path = homedir+"/mteval_test/"
source_seg, ref_seg = download_read_set(mteval_test_path,"en","it","wmt09")
hyp_seg = ref_seg
seg_scores, score_json = measure_ter(hyp_seg,ref_seg)
score = json.loads(score_json)
# Only check a number of score results to verify a validly formatted result is returned
assert score['name'] == 'TER'
assert score['score'] == 0.0
assert len(seg_scores) == 3027

isodate = date.today().isoformat()
score_filename = "ter_test.csv"
measure_record_ter(hyp_seg,ref_seg,"en","it","wmt09",isodate,"dummy",mteval_test_path,score_filename)
assert os.path.exists(mteval_test_path+score_filename) == 1
assert os.path.exists(mteval_test_path+"en_it/"+isodate+"/wmt09/ter_dummy.en-it") == 1

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()